In [1]:
#Conectarnos a la API de Twitter para recopilar datos.
#utilizar las relaciones tipo 'quién retuitea a quién' del topic Arduino

In [2]:
import tweepy
import pickle
from twitter_secrets import CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Autentificación OK")
except tweepy.TweepError as e:
    print("Falló la autentificación")
    print(e)

Autentificación OK


In [3]:
#recopilación de tweets
#La API nos proporciona hasta 18000 tweets cada 15 minutos (100 tweets para cada una de las 180 peticiones máximo permitidas),

In [4]:
#paginación con objeto cursor: http://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html
for page_id, page in enumerate(tweepy.Cursor(
    api.search,q='arduino',count=100,
    tweet_mode='extended',
    languages = ['en', 'es']).pages()): # process status here
    
    fn = f'page_{page_id:06d}.pkl'
    with open("./pickle_arduino/"+fn, 'wb') as f:
        pickle.dump(page, f)

In [5]:
#paginación con objeto cursor: http://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html
import json
for page_id, page in enumerate(tweepy.Cursor(
    api.search,q='Trump',count=100,
    tweet_mode='extended',
    languages = ['en', 'es']).pages()): # process status here
    
    tweets = [i._json for i in page]
    fn = f'page_{page_id:06d}.json'
    with open("./json_trump/"+fn, 'w') as f:
        json.dump(tweets, f)
    

TweepError: Twitter error response: status code = 429

In [6]:
#necesitaremos unos 100000 tweets

In [7]:
#obtener relaciones

In [48]:
from pathlib import Path
import pickle
import pandas as pd
import numpy as np

def read_pickled_pages(fn):
    pickle_files = Path('./pickle_arduino/').glob(fn)
    statuses = []
    for page_fn in pickle_files:
        #print(page_fn)
        with open(page_fn, 'rb') as f:
            statuses.extend(pickle.load(f))
    return statuses

def read_json_pages(fn):
    json_files = Path('./json_trump/').glob(fn)
    statuses = []
    for page_fn in json_files:
        #print(page_fn)
        with open(page_fn, 'rb') as f:
            statuses.extend(json.load(f))
    return statuses

def flatten_dict(nested_json, exclude=['']):
    """Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
            exclude: Keys to exclude from output.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

def statuses_to_pandas(statuses):
    data = []
    for status in statuses:
        data.append(flatten_dict(status._json))
    return pd.DataFrame(data)

def statuses_to_pandas_json(statuses):
    data = []
    for status in statuses:
        data.append(flatten_dict(status))
    return pd.DataFrame(data)

def keep_only_user_id_fields(df):
    def is_user_id_column(c):
        return (('user_id' in c.lower() 
                 or ('user_mentions' in c.lower() and 'id' in c.lower()))
                 and not 'str' in c.lower())
    user_columns = sorted([c for c in df.columns if is_user_id_column(c)], key=len)
    df_nx = df[user_columns] 
    return df_nx


def user_df_to_graph(df_nx):

    graph = set()
    for index, row in df_nx.iterrows():
        user_id = row[0]
        assert(not np.isnan(user_id))
        for other_id in row[1:]:
            try:
                if other_id is not None and not np.isnan(other_id):
                    graph.add((int(user_id), int(other_id)))
            except TypeError:
                pass


    df_graph = pd.DataFrame(graph, columns=['src', 'dest'])
    return df_graph

statuses = read_pickled_pages('*.pkl') #lista con los statuses

In [25]:
len(statuses)

16983

In [26]:
tweets_pd = statuses_to_pandas(statuses)

In [31]:
tweets_pd.describe()

,id,display_text_range_0,display_text_range_1,entities_user_mentions_0_id,entities_user_mentions_0_indices_0,entities_user_mentions_0_indices_1,entities_media_0_id,entities_media_0_indices_0,entities_media_0_indices_1,entities_media_0_sizes_thumb_w,...,quoted_status_user_entities_description_urls_1_indices_0,quoted_status_user_entities_description_urls_1_indices_1,quoted_status_user_entities_description_urls_2_indices_0,quoted_status_user_entities_description_urls_2_indices_1,extended_entities_media_0_additional_media_info_source_user_entities_description_urls_1_indices_0,extended_entities_media_0_additional_media_info_source_user_entities_description_urls_1_indices_1,quoted_status_entities_urls_2_indices_0,quoted_status_entities_urls_2_indices_1,retweeted_status_user_entities_description_urls_3_indices_0,retweeted_status_user_entities_description_urls_3_indices_1
count,1.698300e+04,16983.000000,16983.000000,1.156600e+04,11566.000000,11566.000000,3.005000e+03,3005.000000,3005.000000,3005.000000,...,5.000000,5.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,1.266635e+18,2.988871,128.902196,2.327292e+17,7.545046,17.669635,1.265537e+18,127.996007,150.996007,149.852246,...,99.400000,121.800000,125.0,148.0,119.0,142.0,78.0,101.0,168.0,191.0
std,8.452636e+14,27.076888,57.234558,4.292202e+17,21.950922,22.126279,1.339641e+16,69.632538,69.632538,2.494761,...,32.020306,32.337285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.265262e+18,0.000000,1.000000,6.136000e+03,0.000000,4.000000,8.373249e+17,9.000000,32.000000,91.000000,...,55.000000,78.000000,125.0,148.0,119.0,142.0,78.0,101.0,168.0,191.0
25%,1.265941e+18,0.000000,99.000000,3.388454e+07,3.000000,9.000000,1.265499e+18,87.000000,110.000000,150.000000,...,87.000000,107.000000,125.0,148.0,119.0,142.0,78.0,101.0,168.0,191.0
50%,1.266509e+18,0.000000,139.000000,2.664008e+08,3.000000,13.000000,1.266327e+18,105.000000,128.000000,150.000000,...,94.000000,117.000000,125.0,148.0,119.0,142.0,78.0,101.0,168.0,191.0
75%,1.267456e+18,0.000000,140.000000,4.760788e+09,3.000000,17.000000,1.267187e+18,162.000000,185.000000,150.000000,...,128.000000,151.000000,125.0,148.0,119.0,142.0,78.0,101.0,168.0,191.0
max,1.268122e+18,658.000000,932.000000,1.266856e+18,270.000000,278.000000,1.268122e+18,735.000000,758.000000,150.000000,...,133.000000,156.000000,125.0,148.0,119.0,142.0,78.0,101.0,168.0,191.0


In [32]:
#lista de relaciones id_user (user.id) y en retweeted_status.user.id

In [33]:
relaciones_pd=keep_only_user_id_fields(tweets_pd)

In [34]:
relaciones_pd.describe()

,user_id,in_reply_to_user_id,quoted_status_user_id,retweeted_status_user_id,entities_user_mentions_0_id,entities_user_mentions_1_id,entities_user_mentions_2_id,entities_user_mentions_3_id,entities_user_mentions_4_id,entities_user_mentions_5_id,...,retweeted_status_quoted_status_entities_user_mentions_1_id,retweeted_status_quoted_status_entities_user_mentions_2_id,retweeted_status_quoted_status_entities_user_mentions_3_id,retweeted_status_quoted_status_entities_user_mentions_4_id,retweeted_status_quoted_status_entities_user_mentions_5_id,extended_entities_media_0_additional_media_info_source_user_id,retweeted_status_quoted_status_entities_media_0_source_user_id,retweeted_status_quoted_status_extended_entities_media_0_source_user_id,quoted_status_extended_entities_media_0_additional_media_info_source_user_id,retweeted_status_extended_entities_media_0_additional_media_info_source_user_id
count,1.698300e+04,1.774000e+03,2.450000e+02,9.192000e+03,1.156600e+04,2.095000e+03,9.060000e+02,4.540000e+02,2.220000e+02,1.560000e+02,...,3.000000e+01,9.000000e+00,2.000000e+00,1.0,1.0,7.200000e+01,2.500000e+01,2.500000e+01,2.0,2.000000e+01
mean,4.398314e+17,3.464544e+17,2.267731e+17,2.259187e+17,2.327292e+17,2.235437e+17,2.527618e+17,1.263024e+17,2.037318e+17,1.033356e+17,...,5.279510e+17,6.143496e+08,4.864117e+08,17877351.0,14642896.0,5.446408e+17,1.252922e+17,1.252922e+17,266400754.0,1.523783e+08
std,5.194239e+17,4.995458e+17,4.390404e+17,4.219745e+17,4.292202e+17,4.303091e+17,4.549073e+17,3.373457e+17,4.334782e+17,2.903765e+17,...,5.469750e+17,1.325307e+09,6.585715e+08,NaN,NaN,5.460976e+17,3.497962e+17,3.497962e+17,0.0,1.169845e+08
min,1.148100e+04,6.136000e+03,7.672850e+05,1.148100e+04,6.136000e+03,5.279000e+03,7.415430e+05,1.334800e+04,6.296300e+04,6.296300e+04,...,9.003112e+06,1.361329e+07,2.073130e+07,17877351.0,14642896.0,3.331578e+07,2.168990e+07,2.168990e+07,266400754.0,3.835592e+07
25%,2.840686e+08,1.495473e+08,2.664008e+08,3.388454e+07,3.388454e+07,9.281910e+07,9.178113e+07,1.186140e+08,5.207822e+07,1.152518e+08,...,4.639309e+07,2.854242e+07,2.535715e+08,17877351.0,14642896.0,2.218239e+08,3.835592e+07,3.835592e+07,266400754.0,3.835592e+07
50%,3.178720e+09,1.669807e+09,2.664008e+08,2.664008e+08,2.664008e+08,2.664008e+08,2.664008e+08,3.194245e+08,3.383353e+08,2.664008e+08,...,4.081682e+17,2.664008e+08,4.864117e+08,17877351.0,14642896.0,7.710075e+17,3.835592e+07,3.835592e+07,266400754.0,1.523783e+08
75%,1.000303e+18,9.192716e+17,2.922268e+09,4.317986e+09,4.760788e+09,4.193540e+09,4.338453e+09,1.859962e+09,2.758682e+09,9.059965e+08,...,1.139636e+18,2.664008e+08,7.192519e+08,17877351.0,14642896.0,1.155088e+18,2.412613e+08,2.412613e+08,266400754.0,2.664008e+08
max,1.267868e+18,1.267796e+18,1.258285e+18,1.266856e+18,1.266856e+18,1.266584e+18,1.256350e+18,1.247259e+18,1.206979e+18,1.232407e+18,...,1.176820e+18,4.134255e+09,9.520920e+08,17877351.0,14642896.0,1.254139e+18,1.147923e+18,1.147923e+18,266400754.0,2.664008e+08


In [35]:
grafo = user_df_to_graph(relaciones_pd)

In [36]:
grafo

,src,dest
0,2933209888,33884545
1,1119982272206315520,1139636315031404544
2,3640559832,266400754
3,24969935,101584084
4,1142058348,1607992999
...,...,...
14372,20531864,46096376
14373,4710601,13475312
14374,2616208171,70266297
14375,2444040222,2372055768


In [37]:
grafo.describe()

,src,dest
count,1.437700e+04,1.437700e+04
mean,3.675073e+17,1.959440e+17
std,5.028204e+17,4.058739e+17
min,1.148100e+04,3.380000e+02
25%,1.842114e+08,3.388454e+07
50%,1.668358e+09,2.664008e+08
75%,9.318553e+17,2.876785e+09
max,1.267846e+18,1.267796e+18


In [39]:
#guardo el grafo en formato csv
grafo.to_csv('grafo_arduino.csv', sep=' ', index=False, header=False)

In [42]:
statuses_json = read_json_pages('*.json') #lista con los statuses

In [46]:
len(statuses_json)

18000

In [49]:
tweets_json_pd = statuses_to_pandas_json(statuses_json)

In [50]:
tweets_json_pd.describe()

,id,display_text_range_0,display_text_range_1,entities_hashtags_0_indices_0,entities_hashtags_0_indices_1,entities_hashtags_1_indices_0,entities_hashtags_1_indices_1,entities_hashtags_2_indices_0,entities_hashtags_2_indices_1,entities_hashtags_3_indices_0,...,retweeted_status_entities_media_1_sizes_thumb_w,retweeted_status_entities_media_1_sizes_thumb_h,retweeted_status_entities_media_1_sizes_large_w,retweeted_status_entities_media_1_sizes_large_h,retweeted_status_entities_media_1_sizes_medium_w,retweeted_status_entities_media_1_sizes_medium_h,retweeted_status_entities_media_1_sizes_small_w,retweeted_status_entities_media_1_sizes_small_h,retweeted_status_entities_media_1_source_status_id,retweeted_status_entities_media_1_source_user_id
count,1.800000e+04,18000.000000,18000.000000,1679.000000,1679.000000,569.000000,569.000000,253.000000,253.000000,133.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000000e+00,1.000000e+00
mean,1.268132e+18,3.131667,132.400444,68.645622,80.519952,95.170475,106.316344,112.162055,123.525692,127.421053,...,150.0,150.0,1700.0,1133.0,1200.0,800.0,680.0,453.0,1.268041e+18,7.476661e+17
std,5.685625e+11,17.561784,47.617262,58.242456,58.821608,58.114543,58.931135,59.024489,59.840432,59.760534,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.268131e+18,0.000000,6.000000,0.000000,3.000000,5.000000,11.000000,13.000000,25.000000,26.000000,...,150.0,150.0,1700.0,1133.0,1200.0,800.0,680.0,453.0,1.268041e+18,7.476661e+17
25%,1.268131e+18,0.000000,112.000000,21.000000,35.000000,57.000000,67.000000,78.000000,89.000000,88.000000,...,150.0,150.0,1700.0,1133.0,1200.0,800.0,680.0,453.0,1.268041e+18,7.476661e+17
50%,1.268132e+18,0.000000,140.000000,61.000000,74.000000,82.000000,95.000000,98.000000,109.000000,110.000000,...,150.0,150.0,1700.0,1133.0,1200.0,800.0,680.0,453.0,1.268041e+18,7.476661e+17
75%,1.268132e+18,0.000000,140.000000,93.000000,105.000000,118.000000,130.000000,128.000000,135.000000,169.000000,...,150.0,150.0,1700.0,1133.0,1200.0,800.0,680.0,453.0,1.268041e+18,7.476661e+17
max,1.268133e+18,666.000000,918.000000,905.000000,910.000000,313.000000,321.000000,322.000000,331.000000,292.000000,...,150.0,150.0,1700.0,1133.0,1200.0,800.0,680.0,453.0,1.268041e+18,7.476661e+17


In [51]:
relaciones_json_pd=keep_only_user_id_fields(tweets_json_pd)
relaciones_json_pd.describe()

,user_id,in_reply_to_user_id,quoted_status_user_id,retweeted_status_user_id,entities_user_mentions_0_id,entities_user_mentions_1_id,entities_user_mentions_2_id,entities_user_mentions_3_id,entities_user_mentions_4_id,entities_user_mentions_5_id,...,retweeted_status_quoted_status_entities_user_mentions_3_id,retweeted_status_quoted_status_entities_user_mentions_4_id,retweeted_status_quoted_status_entities_user_mentions_5_id,retweeted_status_quoted_status_entities_user_mentions_6_id,extended_entities_media_0_additional_media_info_source_user_id,retweeted_status_quoted_status_entities_media_0_source_user_id,retweeted_status_quoted_status_extended_entities_media_0_source_user_id,quoted_status_extended_entities_media_0_additional_media_info_source_user_id,retweeted_status_extended_entities_media_0_additional_media_info_source_user_id,retweeted_status_quoted_status_extended_entities_media_0_additional_media_info_source_user_id
count,1.800000e+04,2.057000e+03,3.230000e+02,1.420800e+04,1.643000e+04,2.414000e+03,9.810000e+02,4.210000e+02,2.070000e+02,1.250000e+02,...,2.000000e+00,1.0,1.0,1.0,2.960000e+02,1.240000e+02,1.240000e+02,5.000000e+00,4.600000e+01,5.000000e+00
mean,4.674965e+17,2.983749e+17,2.508349e+17,2.888775e+17,2.858426e+17,2.361715e+17,2.093107e+17,2.463386e+17,2.837787e+17,3.667609e+17,...,5.310788e+07,264739059.0,200476117.0,284597363.0,5.467286e+17,1.507859e+17,1.507859e+17,4.471187e+17,2.866723e+17,2.174761e+17
std,5.267312e+17,4.786269e+17,4.423722e+17,4.687897e+17,4.678406e+17,4.359240e+17,4.082627e+17,4.417055e+17,4.569685e+17,5.043784e+17,...,4.016619e+06,NaN,NaN,NaN,5.464505e+17,3.400483e+17,3.400483e+17,6.126201e+17,4.714427e+17,4.862913e+17
min,3.862300e+04,1.200000e+01,8.070950e+05,1.688000e+03,1.200000e+01,1.688000e+03,1.200000e+01,7.592510e+05,7.592510e+05,9.390910e+05,...,5.026770e+07,264739059.0,200476117.0,284597363.0,1.917731e+06,1.610658e+07,1.610658e+07,1.737689e+07,7.592510e+05,2.507388e+07
25%,4.065005e+08,2.507388e+07,2.507388e+07,7.394900e+07,5.700969e+07,2.507388e+07,2.507388e+07,2.507388e+07,2.507388e+07,4.769193e+07,...,5.168779e+07,264739059.0,200476117.0,284597363.0,1.947164e+09,1.952885e+08,1.952885e+08,2.824160e+07,7.471547e+08,2.605916e+07
50%,3.234065e+09,4.198499e+08,2.568816e+08,9.702073e+08,8.570017e+08,1.071833e+08,2.175432e+08,2.175432e+08,4.328953e+08,4.990740e+08,...,5.310788e+07,264739059.0,200476117.0,284597363.0,8.063763e+17,1.915549e+09,1.915549e+09,1.334569e+09,2.800581e+09,2.605916e+07
75%,1.021363e+18,8.061771e+17,3.653394e+17,8.000867e+17,7.863099e+17,3.386462e+09,2.858648e+09,4.807056e+09,8.025209e+17,8.222157e+17,...,5.452796e+07,264739059.0,200476117.0,284597363.0,1.087380e+18,2.923527e+09,2.923527e+09,1.087380e+18,7.138393e+17,1.334569e+09
max,1.268130e+18,1.268129e+18,1.267976e+18,1.268029e+18,1.268129e+18,1.268093e+18,1.267623e+18,1.266580e+18,1.267573e+18,1.267573e+18,...,5.594805e+07,264739059.0,200476117.0,284597363.0,1.265762e+18,1.249247e+18,1.249247e+18,1.148213e+18,1.243560e+18,1.087380e+18


In [52]:
grafo_json = user_df_to_graph(relaciones_json_pd)
grafo_json.describe()
grafo_json.to_csv('grafo_json_trump.csv', sep=' ', index=False, header=False)

In [53]:
len(grafo_json)

23706

In [54]:
len(grafo)

14377

In [55]:
grafo

,src,dest
0,2933209888,33884545
1,1119982272206315520,1139636315031404544
2,3640559832,266400754
3,24969935,101584084
4,1142058348,1607992999
...,...,...
14372,20531864,46096376
14373,4710601,13475312
14374,2616208171,70266297
14375,2444040222,2372055768


In [56]:
grafo_json

,src,dest
0,2970569441,326255267
1,793965581385011200,15429807
2,1221366000752324608,37034483
3,1020987296,591505507
4,1734300049,25073877
...,...,...
23701,17391764,254117355
23702,1958521082,17980523
23703,1237091453534429184,1167415560
23704,2266797787,496120047


In [57]:
#Parte 2 Analizar la red

In [58]:
#Limpieza de datos

In [59]:
#Análisis de la red

In [60]:
#Análisis de los nodos

In [61]:
#Detección de comunidades